# BERTopic Demo

The purpose of this notebook is to demonstrate topic modelling and related concepts using [BERTopic](https://maartengr.github.io/BERTopic/index.html)

BERTopic is a widely using topic-modelling library that follows a modularizable five-step process, making it flexible and useful for many different topic-modelling circumstances.

##  Imports

In [29]:
from datasets import load_dataset
import pandas as pd

from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer

## Dataset

We will use the [stanfordnlp/web_questions](https://huggingface.co/datasets/stanfordnlp/web_questions) dataset for our modelling. 

This dataset consists of a set of questions and answers sourced from web forums circa 2013. 

In [30]:
data = load_dataset("stanfordnlp/web_questions")

train = pd.DataFrame(data["train"])
test = pd.DataFrame(data["test"])

#concatenate comma-separated answers into single string
train["answers"] = train["answers"].apply(lambda x: ','.join(x))
test["answers"] = test["answers"].apply(lambda x: ','.join(x))

The BERTopic API requires a list of strings as training data.

Let's concatenate the `url`, `question`, and `answers` columns into a single `document` column for use as our training data.

In [31]:
train["document"] = train.apply(lambda row: f"{row["url"]} Q: {row["question"]} A: {row["answers"]}", axis=1)
test["document"] = test.apply(lambda row: f"{row["url"]} Q: {row["question"]} A: {row["answers"]}", axis=1)

train["document"].iloc[0]

'http://www.freebase.com/view/en/justin_bieber Q: what is the name of justin bieber brother? A: Jazmyn Bieber,Jaxon Bieber'

## BERTopic Pipeline

The BERTopic pipeline is composed of five steps:
1. **Embedding** - transform the raw text into floating point numbers that the computer can operate on
2. **Dimensionality Reduction** - Try to represent the floating point arrays in a smaller size so that they take up less memory and are easier to work with
3. **Clustering** - Group the embedded documents by similarities in semantics, text, etc. This will help us discover patterns in the training data.
4. **Tokenize** - Split up longer texts into chunks i.e. tokens. The chunking (tokenization) strategy used here affects how fine-grained the topic results are.
5. **Representation**  - Create a human-friendly display of the shared topics that were discovered in the text. By default this makes keywords.

Each step has multiple choices of algorithm; for example, swapping out one dimensionality reduction algorithm for another might result in different end results. For the purpose of this demonstration I'll use the default algorithms to keep it simple.

Let's walk through each step of the pipeline.

**Note**: The algorithms used here are all CPU based and can work without a GPU. For this demonstration, I'll use CPU versions of the algorithms so that this notebook is portable across environments. In the event that a GPU is available, the code can be modified to use GPU acceleration by following the instructions in the [BERTopic documentation here](https://maartengr.github.io/BERTopic/faq.html#can-i-use-the-gpu-to-speed-up-the-model).

### Embedding

Convert the text into floating-point numerical arrays. By converting text into numbers we can apply mathematical operations and discover mathematical patterns within the embeddings. These mathematical patterns and similarities will ultimately be used to group documents by topic.

The `SentenceTransformers` library is a good default here, balancing speed and quality.

In [32]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

### Dimensionality Reduction

The embeddings created in the previous step can result in high-dimension matrices that have difficulty fitting into memory and affect performance. Dimensionality reduction algorithms like `UMAP()` can shrink the size of the matrices while maintaining most of the embedded information.

The most important parameter in `UMAP()` is `n_neighbors` which controls how many surrounding data points to use when estimating the structure of the data. Higher values create a more global view of the data, at the cost of local detail.

In [ ]:
umap_model = UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
    metric='cosine',
    random_state=42
)

### Clustering

This is the core step in the pipeline, because this step is the one that starts grouping documents by shared topic. These documents groups (clusters) are what the topic representation step will use as a basis to extract keywords/topics. 

The three most important parameters for `HDBSCAN()` are `min_cluster_size`, `min_samples`, and `cluster_selection_method`. These values affect how large and spread-out the resulting clusters are.

Higher values of `min_cluster_size` will create larger clusters that cover more documents but have more general high-level topics.
The value of `min_samples` defaults to `min_cluster_size` but can be set independently to further alter the resulting clusters.
The `cluster_selection_method` has multiple options, but the two most commonly used are `eom` and `leaf`. `eom` has a tendency to create one or two large clusters and several smaller clusters, while `leaf` tries to make evenly sized clusters. 

Experiment with combinations of the three parameters to see what fits your use case.

You can read more about the three parameters and their effects here:<br>
[min_samples and min_cluster_size](https://maartengr.github.io/BERTopic/getting_started/parameter%20tuning/parametertuning.html#hdbscan)<br>
[HDBSCAN](https://hdbscan.readthedocs.io/en/latest/parameter_selection.html)

In [ ]:
hdbscan_model = HDBSCAN(
    min_cluster_size=15,
    min_samples=15,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True
)

### Vectorization

This will tokenize (that is, chunk) texts. 

The most important parameter for `CountVectorizer()` is `ngram_range` which sets how many words each token has; choose a value that makes sense for your data. For example, if you are interested in finding instances of "steak restaurant" in your data, `ngram_range` should allow for tokens up to 2 words long, since "steak restaurant" has two words.

`stop_words` are words that appear frequently in your documents and you want to ignore. By default it uses the `sklearn` english stopwords, [but those have issues](https://scikit-learn.org/stable/modules/feature_extraction.html#stop-words), so you are advised to use another set of stopwords. [NLTK stopwords are a common alternative](https://www.geeksforgeeks.org/nlp/removing-stop-words-nltk-python/).

[You can read more about the vectorizer parameters and algorithms here](https://maartengr.github.io/BERTopic/getting_started/vectorizers/vectorizers.html)

In [35]:
vectorizer_model = CountVectorizer(ngram_range=(1,3), stop_words="english")

### Representation

This step examines each cluster and extracts keywords that are representative of each cluster. These keywords are used to generate the topics for each cluster. 

The default c-TF-IDF algorithm is an innovative algorithm created by the BERTopic developers, and the distinguishing feature of the library. c-TF-IDF differs from standard TF-IDF by treating each cluster of documents as a single merged document and performing TF-IDF on that document. The most frequent terms in that merged document would then be the same as the topics.

The documentation explains it well: _"When you apply TF-IDF as usual on a set of documents, what you are doing is comparing the importance of words between documents. Now, what if, we instead treat all documents in a single category (e.g., a cluster) as a single document and then apply TF-IDF? The result would be importance scores for words within a cluster. The more important words are within a cluster, the more it is representative of that topic. In other words, if we extract the most important words per cluster, we get descriptions of **topics**! This model is called **class-based TF-IDF**"_

There are two boolean parameters that affect how much BERTopic focuses on stopwords and ignores over-frequent words, `bm25_weighting` and `reduce_frequent_words` [(documentation)](https://maartengr.github.io/BERTopic/getting_started/ctfidf/ctfidf.html#parameters). `bm25_weighting` increases the weight of stopwords such that they are more likely to be ignored when generating keywords. `reduce_frequent_words` does just as it says, in that if a word appears too frequently it will be treated as noise instead of as a keyword.

[You can read more about the c-TF-IDF algorithm here](https://maartengr.github.io/BERTopic/algorithm/algorithm.html#5-topic-representation)

In [36]:
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True, bm25_weighting=True)

## Training

Now that all the individual pipeline components have been prepared, we can begin training.

Training is as simple as passing the pipeline components to the `BERTopic()` constructor and running `fit_transform()`.

In [37]:
topic_model = BERTopic(
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  ctfidf_model=ctfidf_model
)

topics, probs = topic_model.fit_transform(train["document"])

## Visualizing Results

Now that we've clustered document together, let's see what distinct topics BERTopic found. There are several visualization methods available, all with their own level of detail and perspective. 

[You can read more about the different visualizations here](https://maartengr.github.io/BERTopic/getting_started/visualization/visualize_topics.html)

I will demonstrate two visualizations: Intertopic Distance and Hiearchical Labelling

### Intertopic Distance

This is a commonly used visualization, it lets you see how big each cluster is and how far apart, or distinct, each topic cluster is from others.

Based on the results of this visualization you can determine if the topics are as distinct as you want them to be, and if the clusters are as fine-grained or general as you would like. This graph can help you visualize the effects of altering the clustering or dimensionality reduction parameters.

[Documentation](https://maartengr.github.io/BERTopic/getting_started/visualization/visualize_topics.html#visualize-topics)

In [38]:
topic_model.visualize_topics()

### Hiearchical Labelling

Topics can often be divided into overarching parent topics and child topics. Hiearchical labelling can help us visualize the parent-child relationship between topics.

[Documentation](https://maartengr.github.io/BERTopic/getting_started/visualization/visualize_hierarchy.html)

In [39]:
hierarchical_topics = topic_model.hierarchical_topics(train["document"])

topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

100%|██████████| 60/60 [00:00<00:00, 304.99it/s]


## Outliers

Most datasets have some documents that do not fit neatly into any topic, these documents are dubbed outliers. Outliers are natural and forcing outliers into clusters risks introducing noise. 

That being said, sometimes you really do need to reduce outliers. BERTopic has an aptly named `reduce_outliers()` function for this very purpose.

`reduce_outliers()` has four main strategies - probabilities, topic distributions, c-TF-IDF, and embeddings. [You can read about each strategy's characteristics here](https://maartengr.github.io/BERTopic/getting_started/outlier_reduction/outlier_reduction.html).

### Counting Outliers

Before we begin outlier reduction, it helps to see how many outliers there are. The -1 topic is the outlier topic, you can see the number of documents in this topic under the `Count` column. As you can see from the `Representation` column, the keywords are unrelated and random since it's a catch-all topic.

In [40]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,807,-1_team_national_london_potter,"[team, national, london, potter, harry potter,...",[http://www.freebase.com/view/en/new_zealand Q...
1,0,349,0_plays_movies_played_star,"[plays, movies, played, star, voice, movie, gr...",[http://www.freebase.com/view/en/taylor_lautne...
2,1,174,1_school_college_university_high school,"[school, college, university, high school, hig...",[http://www.freebase.com/view/en/barack_obama ...
3,2,173,2_live_born_cooper_tupac,"[live, born, cooper, tupac, shakira, selena, s...",[http://www.freebase.com/view/en/selena_gomez ...
4,3,148,3_married_marry_jackson_pryor,"[married, marry, jackson, pryor, kardashian, h...",[http://www.freebase.com/view/en/kennedy_famil...
...,...,...,...,...,...
57,56,17,56_senators_senator_illinois current_en illino...,"[senators, senator, illinois current, en illin...",[http://www.freebase.com/view/en/illinois Q: w...
58,57,16,57_george bush_ronald_reagan_en ronald_reagan_...,"[george bush, ronald_reagan, en ronald_reagan,...",[http://www.freebase.com/view/en/george_w_bush...
59,58,16,58_wales_scotland_northern ireland_northern ir...,"[wales, scotland, northern ireland, northern i...",[http://www.freebase.com/view/en/united_kingdo...
60,59,15,59_pakistan_afghanistan_en pakistan_view en pa...,"[pakistan, afghanistan, en pakistan, view en p...",[http://www.freebase.com/view/en/pakistan Q: w...


### Outlier Reduction

Now that we've counted our outliers, let's reduce them. The c-TF-IDF strategy is the default, and works well enough for most use cases, so let's try that.

In [ ]:
new_topics = topic_model.reduce_outliers(train["document"], topics, strategy="c-tf-idf")

### Update Topics

After reducing outliers, we need to update the topic representations to account for these larger clusters. 

**Do not use topic reduction or topic merging techniques after updating topics, or else it will interfere with topic mapping**

In [42]:
topic_model.update_topics(train["document"], topics=new_topics)

2025-11-19 14:36:08,688 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


### Visualize Updated Topics

After reducing outliers and updating topics, the -1 topic has disappeared.

In [45]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,352,0_who_plays_played_movies,"[who, plays, played, movies, in, the, star, on...",[http://www.freebase.com/view/en/taylor_lautne...
1,1,207,1_school_university_college_go,"[school, university, college, go, high, obama,...",[http://www.freebase.com/view/en/barack_obama ...
2,2,192,2_live_where_born_from,"[live, where, born, from, www, freebase, http,...",[http://www.freebase.com/view/en/selena_gomez ...
3,3,169,3_married_who_jackson_is,"[married, who, jackson, is, marry, freebase, w...",[http://www.freebase.com/view/en/kennedy_famil...
4,4,124,4_newton_did_franklin_scientist,"[newton, did, franklin, scientist, benjamin_fr...",[http://www.freebase.com/view/en/thomas_edison...
...,...,...,...,...,...
56,56,27,56_illinois_senators_senator_missouri,"[illinois, senators, senator, missouri, georgi...",[http://www.freebase.com/view/en/illinois Q: w...
57,57,31,57_ronald_reagan_reagan_george_president,"[ronald_reagan, reagan, george, president, bus...",[http://www.freebase.com/view/en/george_w_bush...
58,58,25,58_scotland_kingdom_wales_united_kingdom,"[scotland, kingdom, wales, united_kingdom, ire...",[http://www.freebase.com/view/en/united_kingdo...
59,59,22,59_pakistan_afghanistan_laden_bin,"[pakistan, afghanistan, laden, bin, osama_bin_...",[http://www.freebase.com/view/en/pakistan Q: w...


## Inference

The fitted topic model can also predict the topics of never-before-seen documents using the `transform()` method.

Keep in mind that even after outlier reduction, some documents might still be classified as outliers during inference.

### Predict

In [46]:
topics, probs = topic_model.transform(test["document"])

### Predicted Topics

In [53]:
n = len(topics)

for i in range(n):
    topic_name = ''.join(topic_model.get_topic_info(topics[i])["Name"].values) or "Outlier"

    print(f"Document: {test.iloc[i]['document']}")
    print(f"Predicted Topic: {topic_name}")
    print()

Document: http://www.freebase.com/view/en/jamaica Q: what does jamaican people speak? A: Jamaican Creole English Language,Jamaican English
Predicted Topic: 12_jamaica_peso_dominican_dominican_republic

Document: http://www.freebase.com/view/en/james_k_polk Q: what did james k polk do before he was president? A: Lawyer
Predicted Topic: Outlier

Document: http://www.freebase.com/view/en/oregon_ducks Q: what is the oregon ducks 2012 football schedule? A: University of Oregon
Predicted Topic: Outlier

Document: http://www.freebase.com/view/en/ken_barlow Q: who plays ken barlow in coronation street? A: Tony Warren
Predicted Topic: 0_who_plays_played_movies

Document: http://www.freebase.com/view/en/chiune_sugihara Q: what happened after mr. sugihara died? A: Yaotsu
Predicted Topic: Outlier

Document: http://www.freebase.com/view/en/wolfgang_amadeus_mozart Q: who did mozart write his four horn concertos for? A: wolfgang amadeus mozart used story by pierre beaumarchais
Predicted Topic: 8_musi

## Conclusion

There's a lot more to topic-modelling and BERTopic than what is demonstrated here. Look around and see what you discover!